# Development Notebook 

## Post Processing

---


### WSE creation using rasmapper 

 1. Pull results and other data from s3 
 - Create xml input file
 - Call `Mapper64\RasComputeMaps`
 - Copy results to s3 output location


In [ ]:
#! aws s3 cp s3://probmodelingrepository/Natchitoches-LA/ProductionRuns/outputs/Pluvial/E006/RedRiverCWMS_Trim.p01.hdf processing/RedRiverCWMS_Trim.p01.hdf
#! aws s3 cp s3://probmodelingrepository/Natchitoches-LA/ProductionRuns/inputs/Pluvial/Terrain/topo19.hdf processing/topo19.hdf
#! aws s3 cp s3://probmodelingrepository/Natchitoches-LA/ProductionRuns/inputs/Pluvial/Terrain/topo19.topo_19.tif processing/topo19.topo_19.tif
#! aws s3 cp s3://probmodelingrepository/Natchitoches-LA/ProductionRuns/inputs/Pluvial/Terrain/topo19.vrt processing/topo19.vrt
#! aws s3 cp s3://probmodelingrepository/Natchitoches-LA/ProductionRuns/inputs/Pluvial/Projection.prj processing/Projection.prj

In [ ]:
import subprocess
import os

##### Project specific arguments

In [ ]:
terrain = "topo19"
project = "RedRiverCWMS_Trim"
projection = "Projection.prj"
planID = '01'

planFile = "{}.p{}.hdf".format(project, planID)

##### Need to verify results consistent with RasMapper (e.g. hybrid interpolation approach used)

In [ ]:
def write_rasmap_file(projection, planFile, terrain):
    xmldata =  """<RASMapper>\n  <RASProjectionFilename Filename=".\{0}" />
      <Results>
        <Layer Type="RASResults" Filename=".\{1}">
          <Layer Name="elevation" Type="RASResultsMap">
          <MapParameters MapType="elevation" OutputMode="Stored Current Terrain" Terrain="{2}" ProfileIndex="2147483647" />
          </Layer>
        </Layer>
      </Results>
      <Terrains>
        <Layer Name="{2}" Type="TerrainLayer" Filename=".\{2}.hdf">
          <ResampleMethod>near</ResampleMethod>
        </Layer>
      </Terrains>
    <RenderMode>hybrid</RenderMode>
    <MarksWarpMethod>False</MarksWarpMethod>\n</RASMapper>""".format(projection, planFile, terrain)
    return xmldata

def check_map_created(pipe_text):
    if pipe_text.split('\r\n')[-2] !='Completed storing 1 results map layer':
        print(pipe_text)
    else:
        return True
    
def rename_output(project, wkdir=r"C:\Users\Administrator\Desktop\processing"):
    # output_dir will be automatically created and named using the plan name
    output_dir_name = [d for d in os.listdir(wkdir) if os.path.isdir(os.path.join(wkdir, d))][0]
    output_dir_path = os.path.join(wkdir,'{}'.format(output_dir_name))

    # may want to add an assert here or in the main() call.
    files =  os.listdir(output_dir_path)
    
    for rawFilename in files:
        updateFilename = 'WSE_{}_{}.{}'.format(project, output_dir_name, rawFilename.split('.')[-1])
        processing_dirs = os.listdir(wkdir) 

        rasmap_output_file = os.path.join(wkdir, output_dir_name, rawFilename)
        s3_output_file     = os.path.join(wkdir, output_dir_name, updateFilename)

        os.rename(rasmap_output_file, s3_output_file)

##### Write xml input file

In [ ]:
rasmap_xml = write_rasmap_file(projection, planFile, terrain)
with open('processing/{}.rasmap'.format(project), 'w') as f: f.write(rasmap_xml)

##### Create compute argument strings

In [ ]:
f1 = r"C:\Users\Administrator\Desktop\processing\{}.rasmap".format(project)
f2 = r"C:\Users\Administrator\Desktop\processing\{}".format(plan)

##### Call RasComputeMaps

In [ ]:
cmd = r"C:\Users\Administrator\Desktop\Mapper64\RasComputeMaps.exe"

pipe = subprocess.Popen([cmd, f1, f2], stdout=subprocess.PIPE)

pipe_text = pipe.communicate()[0].decode("utf-8")
check_map_created(pipe_text)

assert check_map_created(pipe_text), 'Error Creating Map'

##### Rename output, garbage collection

In [ ]:
rename_output(project)

In [ ]:
### todo:

# Copy to s3
# remove local copies/dirs
# add iterator